In [1]:
import pandas as pd
import numpy as np

from datetime import date

### Importando dataset e-commerce

Para mas informacion del dataset, consulte en el siguiente [link](https://www.kaggle.com/datasets/carrie1/ecommerce-data) 

In [2]:
ecommerce = pd.read_csv("data.csv")

In [3]:
ecommerce.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
ecommerce["Quantity"] = ecommerce.apply( lambda r: r["Quantity"] if r["Quantity"] >= 0 else r["Quantity"]*-1 , axis = 1 )
ecommerce["UnitPrice"] = ecommerce.apply( lambda r: r["UnitPrice"] if r["UnitPrice"] >= 0 else r["UnitPrice"]*-1 , axis = 1 )

### cast de fechas y tratamiento de columnas Dias, mes, año

In [5]:
ecommerce.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [6]:
ecommerce["InvoiceDate"] = pd.to_datetime(ecommerce["InvoiceDate"])

In [7]:
ecommerce.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [8]:
ecommerce.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom


In [9]:
ecommerce["year"] = ecommerce["InvoiceDate"].dt.year
ecommerce["month"] = ecommerce["InvoiceDate"].dt.month
ecommerce["day"] = ecommerce["InvoiceDate"].dt.day

In [10]:
ecommerce.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,1


#### Exploracion de paises

In [11]:
ecommerce["Country"].drop_duplicates()

0               United Kingdom
26                      France
197                  Australia
385                Netherlands
1109                   Germany
1236                    Norway
1404                      EIRE
5320               Switzerland
6421                     Spain
6608                    Poland
7134                  Portugal
7214                     Italy
7279                   Belgium
7986                 Lithuania
9783                     Japan
14938                  Iceland
20000          Channel Islands
20017                  Denmark
29732                   Cyprus
30078                   Sweden
31464                  Austria
31982                   Israel
34083                  Finland
38313                  Bahrain
69007                   Greece
69623                Hong Kong
70758                Singapore
72985                  Lebanon
89570     United Arab Emirates
100810            Saudi Arabia
103598          Czech Republic
119191                  Canada
152712  

### Ejercicios

**Ejercicio 1:** Por pais muestre cual fue el año en el que mejor ingreso se registro y su valor, adicional muestre cual fue el año con mas Invoices (Frecuencia) y otra columna con el total.

Nota: Note que el año con mejor ingreso no es lo mismo con el año con mas invoices, por ejemplo en 2019 Honduras en dos invoices se facturo 10,000. sin embargo, en 2020 se realizaron 20 invoices (pueda que en este año el ingreso de esas invoices hayan sumado 7,000 inferior al 2020) 

*recuerde que el ejemplo es inventado*

Ejemplo de como se deberia de ver el dataframe resultante:


In [12]:
pd.DataFrame({
    "country" : ["Honduras"]
    , "YearMejorIngreso" : [2019]
    , "ValorYearMejorIngreso" : [20000]
    , "YearMejorFrecuenciaInvoices": [2020]
    , "CantidadYearMejorFrecuenciaInvoices": [20]
})

,country,YearMejorIngreso,ValorYearMejorIngreso,YearMejorFrecuenciaInvoices,CantidadYearMejorFrecuenciaInvoices
0,Honduras,2019,20000,2020,20


*Answer:*

In [13]:
ecommerce["total_en_miles"] = ecommerce.apply( lambda r: (r['UnitPrice'] * r['Quantity'])/1000 , axis = 1 )

In [14]:
ingresos = ecommerce.groupby( ['Country','year'] , as_index = False ).agg( {'total_en_miles' : ['sum'] } )
ingresos.columns = ingresos.columns.droplevel(1)
ingresos.columns = ['Country','year','Ingreso_en_miles']

ingresos['row'] = ingresos.sort_values( ['Country','Ingreso_en_miles'] , ascending=False ).groupby(['Country']).cumcount()

ingresos = ingresos.loc[ ingresos['row'] == 0 , ['Country','year','Ingreso_en_miles'] ]

ingresos.columns =  ['Country','YearMejorIngreso','ValorYearMejorIngreso (En miles)']

In [15]:
ingresos.shape

(38, 3)

In [16]:
frecuencia = ecommerce.groupby( ['Country','year'] , as_index = False ).agg( { 'InvoiceNo' : ['count'] } )
frecuencia.columns = frecuencia.columns.droplevel(1)
frecuencia.columns = ['Country','year','frecuencia']

frecuencia['row'] = frecuencia.sort_values( ['Country','frecuencia'] , ascending=False ).groupby(['Country']).cumcount() + 1

frecuencia = frecuencia.loc[ frecuencia['row'] == 1 , ['Country','year','frecuencia'] ]

frecuencia.columns = ['Country','YearMejorFrecuenciaInvoices','CantidadYearMejorFrecuenciaInvoices']

In [17]:
frecuencia.shape

(38, 3)

In [18]:
ingresos.merge( frecuencia , on=["Country"] , how="inner" )

,Country,YearMejorIngreso,ValorYearMejorIngreso (En miles),YearMejorFrecuenciaInvoices,CantidadYearMejorFrecuenciaInvoices
0,Australia,2011,138.904750,2011,1224
1,Austria,2011,9.945680,2011,392
2,Bahrain,2011,0.754140,2011,18
3,Belgium,2011,39.671810,2011,1974
4,Brazil,2011,1.143600,2011,32
5,Canada,2011,3.666380,2011,151
6,Channel Islands,2011,20.451060,2011,741
7,Cyprus,2011,12.643650,2011,539
8,Czech Republic,2011,0.945760,2011,30
9,Denmark,2011,17.861040,2011,369


**Ejercicio 2:** Cual es el Top tres por año de los Invoices de lo mas facturado

Ejemplo de como se deberia de ver el dataframe resultante (Recuerde que este es un ejemplo inventado):

In [19]:
pd.DataFrame({
    "Year" : [2019,2019,2019,2020,2020,2020]
    , "Top" : [1,2,3,1,2,3]
    , "InvoiceNo" : [536365, 536999, 693365, 24987, 56832, 254232]    
    , "TotalInvoice": [30000,28000,27900,29000,25000,24600]
})

,Year,Top,InvoiceNo,TotalInvoice
0,2019,1,536365,30000
1,2019,2,536999,28000
2,2019,3,693365,27900
3,2020,1,24987,29000
4,2020,2,56832,25000
5,2020,3,254232,24600


*Answer:*

In [20]:
total = ecommerce.groupby( ["InvoiceNo","year"] , as_index = False ).agg({ "total_en_miles" : ["sum"] })
total.columns = total.columns.droplevel(1)
total.columns = [ "InvoiceNo" , "Year" , "TotalInvoiceEnMiles" ]

In [21]:
total = total.sort_values( ["Year","TotalInvoiceEnMiles"] , ascending=False )

In [22]:
total['Top'] = total.groupby(['Year']).cumcount() + 1

In [23]:
total.loc[ total["Top"] <= 3 , ["Year","Top","InvoiceNo","TotalInvoiceEnMiles"] ]

,Year,Top,InvoiceNo,TotalInvoiceEnMiles
22025,2011,1,581483,168.46960
25895,2011,2,C581484,168.46960
2303,2011,3,541431,77.18360
1645,2010,1,539750,18.74586
674,2010,2,537659,15.88549
652,2010,3,537632,13.54133


In [24]:
total.loc[ total["TotalInvoiceEnMiles"] < 0 , : ]

,InvoiceNo,Year,TotalInvoiceEnMiles,Top


**Ejercicio 3:** Genere el siguiente Dataset para cada uno de los clientes:

In [25]:
pd.DataFrame({
    "CustomerID" : [111,111,111]
    , "YearMonth" : ["2019-1","2019-2","2019-3"]
    , "TotalValorInvoices" : [20000,22000,0]    
    , "TotalCantidadInvoices" : [3,2,0]    
    , "FacturaSiguienteMes": [1,0,1]
})

,CustomerID,YearMonth,TotalValorInvoices,TotalCantidadInvoices,FacturaSiguienteMes
0,111,2019-1,20000,3,1
1,111,2019-2,22000,2,0
2,111,2019-3,0,0,1


*Ten en cuenta lo siguiente:*

Mediante el cross join puedo emular el producto cartesiano de dos vectores para obtener las combinaciones unicas de los valores de cada uno de ellos

In [26]:
df1 = pd.DataFrame( {"year": [2019,2020]} )
df2 = pd.DataFrame( {"Cliente": [111,112,113]} )

df2.merge(df1, how='cross')

,Cliente,year
0,111,2019
1,111,2020
2,112,2019
3,112,2020
4,113,2019
5,113,2020


*Observaciones:*

* TotalValorInvoices: La suma del total facturado por todas sus facturas
* TotalCantidadInvoices: La cantidad de facturas unicas realizadas en ese mes
* FacturaSiguienteMes: Una variable de respuesta que nos dice si al siguiente mes por ese cliente se encontro por lo menos una factura.

Note que por la logica de este dataset el ultimo año-mes encontrado en todo el dataset se va a descartar del resultado final por que no se sabe el valor del siguiente mes. 1: si se encuentra una factura al siguiente mes, 0: no se encuentra nada.

*Answer:*

In [160]:
customer_detail = ecommerce.groupby( ["CustomerID","year","month"] , as_index = False ).agg( { "InvoiceNo" : ["nunique"] , "total_en_miles" : ["sum"] } )
customer_detail.columns = customer_detail.columns.droplevel(1)

In [161]:
customer_detail.columns = ["CustomerID","year","month","TotalCantidadInvoices","TotalValorInvoices"]

El primer paso para saber que años meses son los que debemos de calcular las combinaciones unicas y que sean continuas

In [162]:
YearMonthTotal = customer_detail.loc[ : , ["year","month"] ].drop_duplicates().sort_values( ["year","month"] , ascending = True ).copy()

Calculemos un Indice correlativo para esta combinaciones de "Year-Month" (Logica de dimensiones de la clase de BDII)

In [163]:
YearMonthTotal['indexDate'] = range(0,YearMonthTotal.shape[0])

Como podemos observar tenemos 13 meses desde diciembre del 2010 has diciembre del 2011

In [164]:
YearMonthTotal

,year,month,indexDate
1,2010,12,0
0,2011,1,1
13,2011,2,2
15,2011,3,3
3,2011,4,4
18,2011,5,5
4,2011,6,6
25,2011,7,7
5,2011,8,8
11,2011,9,9


Ahora vamos hacer un Join con el dataset *customer_detail* para posteriormente calcular cual es el minimo y el maximo a nivel de mes año

In [165]:
customer_detail = customer_detail.merge( YearMonthTotal , on=["year","month"] , how="inner" )

In [166]:
customer_detail_date_range = customer_detail.groupby( ["CustomerID"] , as_index=False ).agg( { "indexDate" : ["min","max"] } ).copy()

In [167]:
customer_detail_date_range

CustomerID indexDate    
                      min max
0       12346.0         1   1
1       12347.0         0  12
2       12348.0         0   9
3       12349.0        11  11
4       12350.0         2   2
...         ...       ...  ..
4367    18280.0         3   3
4368    18281.0         6   6
4369    18282.0         8  12
4370    18283.0         1  12
4371    18287.0         5  10

[4372 rows x 3 columns]

Observe como para algunos clientes solo tenemos informacion de un mes por lo que para ellos practicamente la variable de respuesta sera cero. para considerarlos en el calculo principal vamos a sumar +1 en el max para obtener el NA que nos dira que no hubo actividad extra

In [168]:
customer_detail_date_range.columns = customer_detail_date_range.columns.droplevel(0)

In [169]:
customer_detail_date_range.columns = ["CustomerID","minIndexDate","maxIndexDate"]

In [170]:
customer_detail_date_range["maxIndexDate"] = customer_detail_date_range.apply( lambda r: r["maxIndexDate"] + 1 , axis = 1)

In [171]:
customer_detail_date_range

,CustomerID,minIndexDate,maxIndexDate
0,12346.0,1,2.0
1,12347.0,0,13.0
2,12348.0,0,10.0
3,12349.0,11,12.0
4,12350.0,2,3.0
...,...,...,...
4367,18280.0,3,4.0
4368,18281.0,6,7.0
4369,18282.0,8,13.0
4370,18283.0,1,13.0


Ahora calculemos todas combinaciones de clientes con año mes.

In [172]:
UniqueCustomers = customer_detail.loc[ : , ["CustomerID"] ].drop_duplicates().copy()

In [173]:
dfPivot = UniqueCustomers.merge( YearMonthTotal.loc[ : , ["indexDate"]] , how="cross" )

In [174]:
dfPivot

,CustomerID,indexDate
0,12346.0,0
1,12346.0,1
2,12346.0,2
3,12346.0,3
4,12346.0,4
...,...,...
56831,18268.0,8
56832,18268.0,9
56833,18268.0,10
56834,18268.0,11


4372 * 13 = 56836 ( Unicos clientes * total meses a evaluar )

ahora eliminemos los extremos en los que sabemos que el cliente no volvio o no tuvo actividad

In [175]:
dfPivot = dfPivot.merge( customer_detail_date_range , on=["CustomerID"] , how="inner"  )

In [176]:
dfPivot

,CustomerID,indexDate,minIndexDate,maxIndexDate
0,12346.0,0,1,2.0
1,12346.0,1,1,2.0
2,12346.0,2,1,2.0
3,12346.0,3,1,2.0
4,12346.0,4,1,2.0
...,...,...,...,...
56831,18268.0,8,7,8.0
56832,18268.0,9,7,8.0
56833,18268.0,10,7,8.0
56834,18268.0,11,7,8.0


In [177]:
dfPivot = dfPivot.loc[ (dfPivot["indexDate"] >= dfPivot["minIndexDate"]) & (dfPivot["indexDate"] <= dfPivot["maxIndexDate"]) , ["CustomerID","indexDate"] ]

In [178]:
dfPivot

,CustomerID,indexDate
1,12346.0,1
2,12346.0,2
13,12347.0,0
14,12347.0,1
15,12347.0,2
...,...,...
56805,18203.0,8
56817,18262.0,7
56818,18262.0,8
56830,18268.0,7


Note que este calculo nos ayuda a rellenar esos meses que estan enmedio de dos transacciones, imagine que algun cliente hizo una compra en enero y luego en julio. gracias a ese calculo ahora podremos tener febrero, marzo, abril, mayo y junio

In [179]:
dfPivot = dfPivot.merge( YearMonthTotal , on=["indexDate"] , how="inner" ).loc[: , ["CustomerID","year","month"] ]

In [180]:
dfPivot = dfPivot.merge( customer_detail , on=["CustomerID","year","month"] , how="left" ).sort_values( ["CustomerID","year","month"] , ascending=True )

El siguiente es un caso de los que se explica que se tuvo que rellenar los meses que no hubo actividad

In [181]:
dfPivot.loc[ dfPivot["CustomerID"] == 18287.0 ]

,CustomerID,year,month,TotalCantidadInvoices,TotalValorInvoices,indexDate
10025,18287.0,2011,5,1.0,0.76528,5.0
12393,18287.0,2011,6,NaN,NaN,NaN
14783,18287.0,2011,7,NaN,NaN,NaN
17166,18287.0,2011,8,NaN,NaN,NaN
19568,18287.0,2011,9,NaN,NaN,NaN
22202,18287.0,2011,10,2.0,1.07200,10.0
24654,18287.0,2011,11,NaN,NaN,NaN


Ahora iniciemos a formatear el dataset final para calcular la variable solicitada

In [182]:
dfPivot['YearMonth'] = dfPivot.apply( lambda r: "{0}-{1}".format( int(r["year"]) , int(r["month"]) ) , axis = 1 )

In [183]:
dfPivot = dfPivot.loc[: , ["CustomerID","YearMonth","TotalCantidadInvoices","TotalValorInvoices"] ].fillna(0)

In [184]:
dfPivot["lead"] = dfPivot.groupby( ["CustomerID"] , as_index = False )["TotalCantidadInvoices"].shift(-1)

Eliminamos los registros que no se pueden evaluar

In [185]:
dfPivot = dfPivot.loc[ ~pd.isnull(dfPivot["lead"]) , :  ]

Finalmente calculamos la variable de respuesta

In [186]:
dfPivot["FacturaSiguienteMes"] = dfPivot.apply( lambda r: 1 if r["lead"] > 0 else 0 , axis = 1 )

In [187]:
dfPivot = dfPivot.loc[ : , ["CustomerID","YearMonth","TotalCantidadInvoices","TotalValorInvoices","FacturaSiguienteMes"] ]

RESULTADO FINAL

In [188]:
dfPivot

,CustomerID,YearMonth,TotalCantidadInvoices,TotalValorInvoices,FacturaSiguienteMes
0,12346.0,2011-1,2.0,154.36720,0
2990,12347.0,2010-12,1.0,0.71179,1
1,12347.0,2011-1,1.0,0.47539,0
1370,12347.0,2011-2,0.0,0.00000,0
3938,12347.0,2011-3,0.0,0.00000,1
...,...,...,...,...,...
12393,18287.0,2011-6,0.0,0.00000,0
14783,18287.0,2011-7,0.0,0.00000,0
17166,18287.0,2011-8,0.0,0.00000,0
19568,18287.0,2011-9,0.0,0.00000,1


DEMOSTRACION CON UN EJEMPLO ALEATORIO

In [189]:
customer_detail.loc[ customer_detail["CustomerID"] == 12347.0 , : ].sort_values(["indexDate"])

,CustomerID,year,month,TotalCantidadInvoices,TotalValorInvoices,indexDate
783,12347.0,2010,12,1,0.71179,0
1,12347.0,2011,1,1,0.47539,1
1731,12347.0,2011,4,1,0.63625,4
2630,12347.0,2011,6,1,0.38252,6
3681,12347.0,2011,8,1,0.58491,8
4661,12347.0,2011,10,1,1.29432,10
6086,12347.0,2011,12,1,0.22482,12


In [190]:
dfPivot.loc[ dfPivot["CustomerID"] == 12347.0 , : ]

,CustomerID,YearMonth,TotalCantidadInvoices,TotalValorInvoices,FacturaSiguienteMes
2990,12347.0,2010-12,1.0,0.71179,1
1,12347.0,2011-1,1.0,0.47539,0
1370,12347.0,2011-2,0.0,0.00000,0
3938,12347.0,2011-3,0.0,0.00000,1
5902,12347.0,2011-4,1.0,0.63625,0
8058,12347.0,2011-5,0.0,0.00000,1
10328,12347.0,2011-6,1.0,0.38252,0
12696,12347.0,2011-7,0.0,0.00000,1
15094,12347.0,2011-8,1.0,0.58491,0
17477,12347.0,2011-9,0.0,0.00000,1


**Ejercicio cuatro:** Cual es el top 3 de los productos que menos ingresos generan por Pais del ultimo año encontrado del dataset.

Ejemplo de como se deberia de ver el dataframe resultante (Recuerde que este es un ejemplo inventado): 

In [27]:
pd.DataFrame({
    "Country" : ["Canada","Canada","Canada","Spain","Spain","Spain"]
    , "Top" : [1,2,3,1,2,3]    
    , "StockCode": [546,6886,665,656,888,668]
    , "Description": ["Product X","Product K","Product J","Product C","Product D","Product U"]
    , "TotalOnInvoices": [2000,2100,4000,3000,3800,3900]
})

,Country,Top,StockCode,Description,TotalOnInvoices
0,Canada,1,546,Product X,2000
1,Canada,2,6886,Product K,2100
2,Canada,3,665,Product J,4000
3,Spain,1,656,Product C,3000
4,Spain,2,888,Product D,3800
5,Spain,3,668,Product U,3900


*Answer:*

In [205]:
summary_country = ecommerce.groupby( ["Country","StockCode","Description"] , as_index=False ).agg({ "total_en_miles" : ["sum"] })
summary_country.columns = summary_country.columns.droplevel(1)
summary_country.columns = ["Country","StockCode","Description","sum_totals"]

In [206]:
summary_country = summary_country.sort_values(["Country","sum_totals"], ascending=True)

In [207]:
summary_country["Top"] = summary_country.groupby(["Country"], as_index=False).cumcount() + 1

In [209]:
summary_country.loc[ summary_country["Top"] <= 3 , ["Country","Top","StockCode","Description","sum_totals"] ]

,Country,Top,StockCode,Description,sum_totals
483,Australia,1,23292,SPACEBOY CHILDRENS CUP,0.00250
52,Australia,2,21208,PASTEL COLOUR HONEYCOMB FAN,0.00468
73,Australia,3,21507,"ELEPHANT, BIRTHDAY CARD,",0.00504
658,Austria,1,21402,RED EGG SPOON,0.00288
659,Austria,2,21403,BLUE EGG SPOON,0.00288
...,...,...,...,...,...
15649,United Kingdom,2,10133,damaged,0.00000
15667,United Kingdom,3,15058A,wet/rusty,0.00000
20716,Unspecified,1,79063C,"RETRO PILL BOX KEY CHAIN,THE KING",0.00019
20526,Unspecified,2,22396,MAGNETS PACK OF 4 RETRO PHOTO,0.00039
